In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import h5py
import keras.backend as K
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Activation
from keras import optimizers, layers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


## Training costs(IM1, IM2, IM3, OS1)

In [2]:
x_1 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_train_scale.csv', 
                    delimiter = ',', usecols = (15,16,17,18), skip_header=2)
y_1o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_train_scale.csv', 
                   delimiter =',', usecols = (29), skip_header=2).reshape (54,1)
x_2 = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv', 
                    delimiter = ',', usecols = (15,16,17,18), skip_header=1)
y_2o = np.genfromtxt ('C:/Users/z5023853/OneDrive - UNSW/Separated dataset_ML/a1_cv_scale.csv',
                      delimiter =',', usecols = (29),skip_header=1).reshape (14,1)

In [3]:
enc = OneHotEncoder ()
enc.fit (y_1o)
y_1 = enc.transform(y_1o).toarray()
y_2 = enc.transform(y_2o).toarray()

In [4]:
y_whole = np.vstack ((y_1, y_2)).astype(np.float32)
x_whole = np.vstack ((x_1, x_2)).astype(np.float32)
x_traind = x_1
x_test = x_2
y_traind = y_1
y_test = y_2
x_train, x_cv, y_train, y_cv = train_test_split (x_1,y_1,test_size=0.13)

In [6]:
num_classes = 4;

model = Sequential ()
model.add (Dense (units = 25, input_dim = 4))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 25))
model.add (layers.Activation ('tanh'))
model.add (Dense (units = 25))
model.add (layers.Activation ('tanh'))
#model.add (Dense (units = 16))
model.add (Dense(num_classes, activation = 'softmax'))

#sgd = optimizer.SGD (lr=0.01, momentum = 0, decay =0, nestrov = False)
model.compile (loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])

History = model.fit(x_train, y_train,
                    batch_size=20,
                    epochs=40,
                    verbose=1, validation_data=(x_cv, y_cv))

print(model.evaluate (x_test, y_test, batch_size = 20, verbose =1))

#help (model.evaluate)
#this is the output units

y_pred_train = model.predict (x_train)
y_pred_cv = model.predict (x_cv)
y_pred_test = model.predict (x_test)
y_pred_whole = np.vstack ((y_pred_train, y_pred_cv, y_pred_test)).astype(np.float32)

Train on 46 samples, validate on 8 samples
Epoch 1/40
46/46 [==============================] - ETA: 0s - loss: 0.6065 - binary_accuracy: 0.7500 - val_loss: 0.6338 - val_binary_accuracy: 0.7500
Epoch 2/40
46/46 [==============================] - ETA: 0s - loss: 0.6029 - binary_accuracy: 0.7500 - val_loss: 0.6342 - val_binary_accuracy: 0.7500
Epoch 3/40
46/46 [==============================] - ETA: 0s - loss: 0.5992 - binary_accuracy: 0.7500 - val_loss: 0.6343 - val_binary_accuracy: 0.7500
Epoch 4/40
46/46 [==============================] - ETA: 0s - loss: 0.5962 - binary_accuracy: 0.7500 - val_loss: 0.6330 - val_binary_accuracy: 0.7500
Epoch 5/40
46/46 [==============================] - ETA: 0s - loss: 0.5937 - binary_accuracy: 0.7500 - val_loss: 0.6328 - val_binary_accuracy: 0.7500
Epoch 6/40
46/46 [==============================] - ETA: 0s - loss: 0.5910 - binary_accuracy: 0.7500 - val_loss: 0.6334 - val_binary_accuracy: 0.7500
Epoch 7/40
46/46 [==============================] - ETA: 

### t(x) for labelling

In [7]:
#save model to json, weights to HDF5
model_json = model.to_json ()
with open ('model_lod_train_cost_a.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_lod_train_cost_a.h5')
print ('Saved model to disk')

Saved model to disk


In [5]:
#load json and create model
json_file = open('model_lod_train_cost_a.json', 'r')
loaded_model_json = json_file.read ()
json_file.close ()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model_lod_train_cost_a.h5')

In [6]:
loaded_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', 
               metrics=['binary_accuracy'])
print(loaded_model.evaluate (x_test, y_test, batch_size = 20, verbose =1))
y_pred_test = loaded_model.predict (x_test)


i = 0;
j = 0;
m = np.zeros((14,4));
for i in range (14):
    max = 0.0000001;
    for j in range (4):
        if y_pred_test.item ((i,j))>=max:
            max = y_pred_test.item ((i,j))
    for j in range (4):
        if y_pred_test.item ((i,j)) ==max:
            np.put(m, [4*i+j], [1])

i = 0;
j = 0;
selected_LOD = np.zeros((14,1));
for i in range (14):
    for j in range (4):
        if m.item((i,j))>0:
            selected_LOD_1 = m.item((i,j))*y_pred_test.item((i,j));
            np.put(selected_LOD, [i], [selected_LOD_1])

i = 0;
j = 0;
cf = np.zeros((14,4))
for i in range (14):
    for j in range (4):
        cf_1 = y_pred_test.item((i,j))-selected_LOD.item((i,0));
        cf_2 = 10*cf_1;
        np.put(cf, [4*i+j], [cf_2])
print (cf)
np.savetxt('y_cost_LODtrain_cost.csv', cf, delimiter=',') 

14/14 [==============================] - ETA: 0s
[0.53890615701675415, 0.75]
[[ 0.         -0.6359686  -0.87405086 -0.01046091]
 [ 0.         -1.02063492 -1.39098153 -0.46050191]
 [-2.12497607 -2.5812678  -2.28365138  0.        ]
 [-2.57761136 -1.12115234  0.         -2.78417408]
 [-0.2498284  -1.41997665 -1.34101525  0.        ]
 [ 0.         -0.6359686  -0.87405086 -0.01046091]
 [-0.2653867  -0.80692634 -0.96196383  0.        ]
 [ 0.         -0.69434687 -0.59401736 -0.56054562]
 [-0.71437687 -1.32666677 -1.48433477  0.        ]
 [-0.48702747 -0.30000538 -0.18057436  0.        ]
 [-2.12497607 -2.5812678  -2.28365138  0.        ]
 [-1.38021305 -1.66737065 -1.63922995  0.        ]
 [-2.24070862 -1.20744228  0.         -1.41716525]
 [-2.25000903 -1.24584019  0.         -1.85253546]]
